In [1]:
"""
OPTIMIZED RANGE BACKTEST - 30-60 DAY HISTORICAL
10:00-11:15 AM RANGE (75 minutes)
20 Contracts | 10pt SL | 100pt TP (10:1 R/R)
✅ Includes realistic slippage & commissions
"""
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta, date
import pytz
from collections import Counter

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"
CONTRACT_ID = "CON.F.US.MNQ.H26"

# OPTIMIZED Strategy parameters (from 31-day grid search)
POSITION_SIZE = 20
FIXED_STOP_LOSS = 10
FIXED_TAKE_PROFIT = 100

# Realism parameters
SLIPPAGE_POINTS = 0.25  # 1 tick for MNQ
COMMISSION_PER_CONTRACT = 0.62  # Per side
POINT_VALUE = 0.5  # $0.50 per point for MNQ

# Range parameters
RANGE_START_HOUR = 10
RANGE_START_MINUTE = 0
RANGE_END_HOUR = 11
RANGE_END_MINUTE = 15

# ==================== API FUNCTIONS ====================
def authenticate():
    resp = requests.post(
        f"{API_BASE}/Auth/loginKey",
        json={"userName": USERNAME, "apiKey": KEY},
        headers={"accept": "text/plain", "Content-Type": "application/json"},
        timeout=15
    )
    return {
        "Authorization": f"Bearer {resp.json()['token']}",
        "accept": "text/plain",
        "Content-Type": "application/json"
    }

def fetch_bars(headers, unit, unit_number, days_back=100):
    """Fetch bars in chunks to get more historical data"""
    all_bars = []
    
    # Fetch in 25-day chunks to stay under 10k bar limit
    chunk_size = 25
    chunks_needed = (days_back + chunk_size - 1) // chunk_size  # Round up
    
    end_time = datetime.now(pytz.UTC)
    
    for i in range(chunks_needed):
        chunk_end = end_time - timedelta(days=i * chunk_size)
        chunk_start = chunk_end - timedelta(days=chunk_size)
        
        payload = {
            "contractId": CONTRACT_ID,
            "live": False,
            "startTime": chunk_start.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "endTime": chunk_end.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "unit": unit,
            "unitNumber": unit_number,
            "limit": 10000,
            "includePartialBar": True
        }
        
        try:
            resp = requests.post(f"{API_BASE}/History/retrieveBars", json=payload, headers=headers, timeout=30)
            bars = resp.json().get("bars", [])
            all_bars.extend(bars)
            print(f"   Fetched chunk {i+1}/{chunks_needed}: {len(bars)} bars")
        except Exception as e:
            print(f"   Warning: Chunk {i+1} failed: {e}")
            continue
    
    return all_bars

def parse_bars_with_ny_time(bars):
    ny_tz = pytz.timezone('America/New_York')
    parsed = []
    
    for bar in bars:
        timestamp = bar['t'].replace('+00:00', 'Z')
        dt_utc = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        dt_utc = pytz.utc.localize(dt_utc)
        dt_ny = dt_utc.astimezone(ny_tz)
        
        bar['dt_ny'] = dt_ny
        bar['date'] = dt_ny.date()
        parsed.append(bar)
    
    return parsed

def get_trading_days(bars_5m):
    """Get unique trading days from the data, excluding weekends and holidays"""
    
    # Major US market holidays to exclude
    EXCLUDED_HOLIDAYS = {
        # 2024 holidays
        date(2024, 1, 1), date(2024, 1, 15), date(2024, 2, 19),
        date(2024, 3, 29), date(2024, 5, 27), date(2024, 6, 19),
        date(2024, 7, 4), date(2024, 9, 2), date(2024, 11, 28),
        date(2024, 12, 25),
        # 2025 holidays
        date(2025, 1, 1), date(2025, 1, 20), date(2025, 2, 17),
        date(2025, 4, 18), date(2025, 5, 26), date(2025, 6, 19),
        date(2025, 7, 4), date(2025, 9, 1), date(2025, 11, 27),
        date(2025, 12, 25),
        # 2026 holidays
        date(2026, 1, 1), date(2026, 1, 19), date(2026, 2, 16),
        date(2026, 4, 3), date(2026, 5, 25), date(2026, 6, 19),
        date(2026, 7, 3), date(2026, 9, 7), date(2026, 11, 26),
        date(2026, 12, 25),
    }
    
    days = set()
    for bar in bars_5m:
        if bar['dt_ny'].hour >= 9 and (bar['dt_ny'].hour > 9 or bar['dt_ny'].minute >= 30):
            bar_date = bar['date']
            
            # Skip weekends
            if bar['dt_ny'].weekday() >= 5:
                continue
            
            # Skip holidays
            if bar_date in EXCLUDED_HOLIDAYS:
                continue
            
            days.add(bar_date)
    
    return sorted(list(days), reverse=True)[:60]

def get_range_for_day(bars_5m, target_date):
    """Get the range high/low for a specific day"""
    range_bars = []
    
    for bar in bars_5m:
        if bar['date'] == target_date:
            bar_time_minutes = bar['dt_ny'].hour * 60 + bar['dt_ny'].minute
            start_time_minutes = RANGE_START_HOUR * 60 + RANGE_START_MINUTE
            end_time_minutes = RANGE_END_HOUR * 60 + RANGE_END_MINUTE
            
            if start_time_minutes <= bar_time_minutes < end_time_minutes:
                range_bars.append(bar)
    
    if not range_bars:
        return None
    
    range_high = max(bar['h'] for bar in range_bars)
    range_low = min(bar['l'] for bar in range_bars)
    
    return {
        'high': range_high,
        'low': range_low,
        'size': range_high - range_low
    }

def simulate_trade_realistic(direction, entry, stop, target, future_bars):
    """Realistic trade simulation with slippage, commissions, and intra-bar logic"""
    
    # Apply slippage to entry
    if direction == 'long':
        entry_with_slippage = entry + SLIPPAGE_POINTS
        stop_with_slippage = stop - SLIPPAGE_POINTS
        target_with_slippage = target - SLIPPAGE_POINTS
    else:
        entry_with_slippage = entry - SLIPPAGE_POINTS
        stop_with_slippage = stop + SLIPPAGE_POINTS
        target_with_slippage = target + SLIPPAGE_POINTS
    
    for bar in future_bars:
        if direction == 'long':
            stop_hit = bar['l'] <= stop_with_slippage
            target_hit = bar['h'] >= target_with_slippage
            
            # If both hit in same bar, check which was closer
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (stop_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (target_with_slippage - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
        
        else:  # SHORT
            stop_hit = bar['h'] >= stop_with_slippage
            target_hit = bar['l'] <= target_with_slippage
            
            if stop_hit and target_hit:
                dist_to_stop = abs(entry_with_slippage - stop_with_slippage)
                dist_to_target = abs(entry_with_slippage - target_with_slippage)
                
                if dist_to_stop <= dist_to_target:
                    stop_hit = True
                    target_hit = False
                else:
                    stop_hit = False
                    target_hit = True
            
            if stop_hit:
                gross_pnl = (entry_with_slippage - stop_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'LOSS', 'pnl': net_pnl, 'exit_price': stop_with_slippage}
            
            if target_hit:
                gross_pnl = (entry_with_slippage - target_with_slippage) * POINT_VALUE * POSITION_SIZE
                commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
                net_pnl = gross_pnl - commissions
                return {'result': 'WIN', 'pnl': net_pnl, 'exit_price': target_with_slippage}
    
    # Trade still open at end of day
    if future_bars:
        exit_price = future_bars[-1]['c']
        if direction == 'long':
            gross_pnl = (exit_price - entry_with_slippage) * POINT_VALUE * POSITION_SIZE
        else:
            gross_pnl = (entry_with_slippage - exit_price) * POINT_VALUE * POSITION_SIZE
        
        commissions = COMMISSION_PER_CONTRACT * POSITION_SIZE * 2
        net_pnl = gross_pnl - commissions
        return {'result': 'OPEN', 'pnl': net_pnl, 'exit_price': exit_price}
    
    return {'result': 'OPEN', 'pnl': 0, 'exit_price': entry_with_slippage}

def backtest_single_day(bars_5m, target_date):
    """Run backtest for a single day"""
    
    # Get range data
    range_data = get_range_for_day(bars_5m, target_date)
    if range_data is None:
        return None
    
    range_high = range_data['high']
    range_low = range_data['low']
    
    # Get bars AFTER the range ends
    bars_after_range = []
    for bar in bars_5m:
        if bar['date'] == target_date:
            bar_time_minutes = bar['dt_ny'].hour * 60 + bar['dt_ny'].minute
            range_end_minutes = RANGE_END_HOUR * 60 + RANGE_END_MINUTE
            
            if bar_time_minutes >= range_end_minutes:
                bars_after_range.append(bar)
    
    bars_after_range = list(reversed(bars_after_range))
    
    if len(bars_after_range) < 5:
        return None
    
    trades = []
    
    # Scan for setups
    for i in range(len(bars_after_range) - 1):
        current_bar = bars_after_range[i]
        next_bar = bars_after_range[i + 1]
        
        closed_above = current_bar['c'] > range_high
        closed_below = current_bar['c'] < range_low
        next_closed_inside = range_low <= next_bar['c'] <= range_high
        
        # SHORT SETUP
        if closed_above and next_closed_inside:
            entry_price = next_bar['c']
            stop_price = entry_price + FIXED_STOP_LOSS
            target_price = entry_price - FIXED_TAKE_PROFIT
            
            future_bars = bars_after_range[i+2:]
            outcome = simulate_trade_realistic('short', entry_price, stop_price, target_price, future_bars)
            
            trades.append({
                'date': target_date,
                'time': next_bar['dt_ny'],
                'type': 'SHORT',
                'entry': entry_price,
                'result': outcome['result'],
                'pnl': outcome['pnl']
            })
        
        # LONG SETUP
        elif closed_below and next_closed_inside:
            entry_price = next_bar['c']
            stop_price = entry_price - FIXED_STOP_LOSS
            target_price = entry_price + FIXED_TAKE_PROFIT
            
            future_bars = bars_after_range[i+2:]
            outcome = simulate_trade_realistic('long', entry_price, stop_price, target_price, future_bars)
            
            trades.append({
                'date': target_date,
                'time': next_bar['dt_ny'],
                'type': 'LONG',
                'entry': entry_price,
                'result': outcome['result'],
                'pnl': outcome['pnl']
            })
    
    return {
        'trades': trades,
        'range_high': range_high,
        'range_low': range_low,
        'range_size': range_data['size']
    }

# ==================== MAIN ====================
def main():
    print("=" * 80)
    print("OPTIMIZED RANGE BACKTEST - 30-60 DAY HISTORICAL")
    print(f"Range: {RANGE_START_HOUR:02d}:{RANGE_START_MINUTE:02d}-{RANGE_END_HOUR:02d}:{RANGE_END_MINUTE:02d} (75 min)")
    print(f"Setup: {POSITION_SIZE} Contracts | {FIXED_STOP_LOSS}pt SL | {FIXED_TAKE_PROFIT}pt TP (10:1 R/R)")
    print(f"Realistic: Slippage {SLIPPAGE_POINTS}pts | Commission ${COMMISSION_PER_CONTRACT*2:.2f}/contract")
    print("=" * 80)
    
    print("\n[1/4] Authenticating...")
    headers = authenticate()
    print("✅ Authenticated")
    
    print("\n[2/4] Fetching 5-minute historical data (in chunks)...")
    bars_5m = fetch_bars(headers, unit=2, unit_number=5, days_back=100)
    print(f"✅ Fetched {len(bars_5m)} total 5-minute bars")
    
    bars_5m = parse_bars_with_ny_time(bars_5m)
    
    print("\n[3/4] Identifying trading days...")
    trading_days = get_trading_days(bars_5m)
    print(f"✅ Found {len(trading_days)} valid trading days (weekdays only, excluding holidays)")
    print(f"   Date Range: {trading_days[-1]} to {trading_days[0]}")
    
    # Show day breakdown
    day_counts = Counter([d.strftime('%A') for d in trading_days])
    print(f"   Breakdown: Mon({day_counts.get('Monday', 0)}) Tue({day_counts.get('Tuesday', 0)}) "
          f"Wed({day_counts.get('Wednesday', 0)}) Thu({day_counts.get('Thursday', 0)}) Fri({day_counts.get('Friday', 0)})")
    
    print("\n[4/4] Running backtest across all days...")
    
    all_trades = []
    daily_results = []
    days_with_setups = 0
    
    for idx, day in enumerate(reversed(trading_days), 1):
        result = backtest_single_day(bars_5m, day)
        
        if result and result['trades']:
            days_with_setups += 1
            daily_pnl = sum(t['pnl'] for t in result['trades'])
            
            daily_results.append({
                'date': day,
                'trades': len(result['trades']),
                'pnl': daily_pnl,
                'range_size': result['range_size']
            })
            
            all_trades.extend(result['trades'])
        
        if idx % 10 == 0:
            print(f"Progress: {idx}/{len(trading_days)} days processed")
    
    print(f"✅ Backtest complete")
    
    # Calculate overall stats
    wins = [t for t in all_trades if t['result'] == 'WIN']
    losses = [t for t in all_trades if t['result'] == 'LOSS']
    open_trades = [t for t in all_trades if t['result'] == 'OPEN']
    
    total_pnl = sum(t['pnl'] for t in all_trades)
    total_win_pnl = sum(t['pnl'] for t in wins)
    total_loss_pnl = abs(sum(t['pnl'] for t in losses))
    
    closed_trades = len(wins) + len(losses)
    win_rate = (len(wins) / closed_trades * 100) if closed_trades > 0 else 0
    profit_factor = (total_win_pnl / total_loss_pnl) if total_loss_pnl > 0 else float('inf')
    expectancy = total_pnl / len(all_trades) if all_trades else 0
    
    # Print results
    print("\n" + "=" * 80)
    print("OVERALL RESULTS")
    print("=" * 80)
    
    print(f"\n📅 Period: {trading_days[-1]} to {trading_days[0]}")
    print(f"   Total Trading Days: {len(trading_days)}")
    print(f"   Days with Setups: {days_with_setups} ({100*days_with_setups/len(trading_days):.1f}%)")
    
    print(f"\n📊 TRADES:")
    print(f"   Total: {len(all_trades)}")
    print(f"   Wins: {len(wins)}")
    print(f"   Losses: {len(losses)}")
    print(f"   Open: {len(open_trades)}")
    print(f"   Win Rate: {win_rate:.1f}%")
    
    print(f"\n💰 PROFITABILITY:")
    print(f"   Total P&L: ${total_pnl:.2f}")
    print(f"   Expectancy: ${expectancy:.2f} per trade")
    pf_str = f"{profit_factor:.2f}" if profit_factor != float('inf') else "∞"
    print(f"   Profit Factor: {pf_str}")
    print(f"   Total Wins $: ${total_win_pnl:.2f}")
    print(f"   Total Losses $: ${total_loss_pnl:.2f}")
    
    print(f"\n📈 DAILY STATS:")
    if daily_results:
        avg_daily_pnl = sum(d['pnl'] for d in daily_results) / len(daily_results)
        avg_trades_per_day = len(all_trades) / len(trading_days)
        avg_range_size = sum(d['range_size'] for d in daily_results) / len(daily_results)
        
        print(f"   Avg P&L per trading day: ${avg_daily_pnl:.2f}")
        print(f"   Avg trades per day: {avg_trades_per_day:.1f}")
        print(f"   Avg range size: {avg_range_size:.1f} points")
        
        # Best and worst days
        best_day = max(daily_results, key=lambda x: x['pnl'])
        worst_day = min(daily_results, key=lambda x: x['pnl'])
        
        print(f"\n   Best Day: {best_day['date']} → ${best_day['pnl']:.2f} ({best_day['trades']} trades)")
        print(f"   Worst Day: {worst_day['date']} → ${worst_day['pnl']:.2f} ({worst_day['trades']} trades)")
    
    # Show daily breakdown
    print("\n" + "=" * 80)
    print("DAILY BREAKDOWN")
    print("=" * 80)
    print(f"\n{'Date':<12}{'Trades':<8}{'Wins':<6}{'Losses':<8}{'P&L':<12}{'Range Size'}")
    print("-" * 80)
    
    for day_result in sorted(daily_results, key=lambda x: x['date'], reverse=True)[:20]:
        day_trades = [t for t in all_trades if t['date'] == day_result['date']]
        day_wins = len([t for t in day_trades if t['result'] == 'WIN'])
        day_losses = len([t for t in day_trades if t['result'] == 'LOSS'])
        
        print(f"{day_result['date']!s:<12}{day_result['trades']:<8}{day_wins:<6}{day_losses:<8}"
              f"${day_result['pnl']:<11.2f}{day_result['range_size']:.1f}pts")
    
    if len(daily_results) > 20:
        print(f"\n... ({len(daily_results) - 20} more days)")
    
    print("\n" + "=" * 80)
    print("✅ BACKTEST COMPLETE")
    print("=" * 80)

if __name__ == "__main__":
    main()

OPTIMIZED RANGE BACKTEST - 30-60 DAY HISTORICAL
Range: 10:00-11:15 (75 min)
Setup: 20 Contracts | 10pt SL | 100pt TP (10:1 R/R)
Realistic: Slippage 0.25pts | Commission $1.24/contract

[1/4] Authenticating...
✅ Authenticated

[2/4] Fetching 5-minute historical data (in chunks)...
   Fetched chunk 1/4: 4645 bars
   Fetched chunk 2/4: 4584 bars
   Fetched chunk 3/4: 0 bars
   Fetched chunk 4/4: 0 bars
✅ Fetched 9229 total 5-minute bars

[3/4] Identifying trading days...
✅ Found 33 valid trading days (weekdays only, excluding holidays)
   Date Range: 2025-12-15 to 2026-02-02
   Breakdown: Mon(7) Tue(7) Wed(7) Thu(5) Fri(7)

[4/4] Running backtest across all days...
Progress: 10/33 days processed
Progress: 20/33 days processed
Progress: 30/33 days processed
✅ Backtest complete

OVERALL RESULTS

📅 Period: 2025-12-15 to 2026-02-02
   Total Trading Days: 33
   Days with Setups: 22 (66.7%)

📊 TRADES:
   Total: 82
   Wins: 11
   Losses: 64
   Open: 7
   Win Rate: 14.7%

💰 PROFITABILITY:
   Tota